# Controlling Output

We can control the output from Claude by doing
1. Message Prefilling 
2. Stop Sequences

In [1]:
import boto3

In [2]:
client = boto3.client("bedrock-runtime", region_name="us-east-2")
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"


def add_user_message(messages, text):
    user_message = {"role": "user", "content": [{"text": text}]}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": [{"text": text}]}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
        },
    }

    if system:
        params["system"] = [{"text": system}]

    response = client.converse(**params)

    return response["output"]["message"]["content"][0]["text"]

## Message Prefilling

By providing the start of an assistant message, we can force the LLM to continue from there. 

![message prefilling](https://everpath-course-content.s3-accelerate.amazonaws.com/instructor%2Fa46l9irobhg0f5webscixp0bs%2Fpublic%2F1748557716%2F05_-_009_-_Controlling_Model_Output_05.1748557716502.png)

In [4]:
messages = []

add_user_message(messages, "Is coffee or tea better for breakfast?")

chat(messages)

'The choice between coffee and tea for breakfast depends on your personal preferences and needs.\n\nCoffee typically contains more caffeine, providing a stronger energy boost to start your day. It also contains antioxidants and may help improve mental alertness.\n\nTea (especially green tea) generally has less caffeine but provides a more gradual energy increase. Many teas offer antioxidants and can provide a gentler start to your morning.\n\nNeither is objectively "better" - it comes down to how your body responds to each and what you enjoy drinking with your morning meal.'

In [3]:
messages = []

add_user_message(messages, "Is coffee or tea better for breakfast?")
add_assistant_message(messages, "Coffee is better because")

chat(messages)

" it keeps you more focused and therefore keeps you energized longer. It takes longer for the caffeine to release into the bloodstream meaning that it won't give you that initial kick tea would, but rather a steadier energy."

In [5]:
messages = []

add_user_message(messages, "Is coffee or tea better for breakfast?")
add_assistant_message(messages, "Tea is better because")

chat(messages)

" it's healthier."

## Stop Sequences

Force Claude to end its response immediately when it encounters a specific text.

![Stop Sequences](https://everpath-course-content.s3-accelerate.amazonaws.com/instructor%2Fa46l9irobhg0f5webscixp0bs%2Fpublic%2F1748557717%2F05_-_009_-_Controlling_Model_Output_14.1748557717103.png)

In [4]:
def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
            "stopSequences": stop_sequences,
        },
    }

    if system:
        params["system"] = [{"text": system}]

    response = client.converse(**params)

    return response["output"]["message"]["content"][0]["text"]

In [7]:
messages = []

add_user_message(messages, "Count from 1 to 10")

chat(messages)

'1, 2, 3, 4, 5, 6, 7, 8, 9, 10'

In [ ]:
messages = []

add_user_message(messages, "Count from 1 to 10")

chat(messages, stop_sequences=["6","5"])

'1, 2, 3, 4, '

## Problem with Default Responses

By default Claude tends to format structured output like:

In [7]:
messages = []

add_user_message(messages, "Generate a very short event bridge rule as json")

text = chat(messages)
text

'```json\n{\n  "source": ["aws.ec2"],\n  "detail-type": ["EC2 Instance State-change Notification"],\n  "detail": {\n    "state": ["running"]\n  }\n}\n```'

This usually has a heading, the markdown formatter and oftentimes an explanation. When we need just the code in a structured format, we can use message prefilling and stop sequences together to achieve our result.

## Structured Data

We can use both together to get a very specific type of structured data. For example, if we want some code snippets directly, we can prefill the assistant message with \`\`\`language-name and end it with a stop sequence of \`\`\`


In [5]:
messages = []

add_user_message(messages, "Generate a very short event bridge rule as json")
add_assistant_message(messages, "```json")

text = chat(messages, stop_sequences=["```"])
text

'\n{\n  "source": ["aws.ec2"],\n  "detail-type": ["EC2 Instance State-change Notification"],\n  "detail": {\n    "state": ["running"]\n  }\n}\n'

In [6]:
# we still have the \n characters but we can remove them using the json loads
import json

json.loads(text.strip())

{'source': ['aws.ec2'],
 'detail-type': ['EC2 Instance State-change Notification'],
 'detail': {'state': ['running']}}

The above technique can also be used for other structured data such as:

- Code snippets (Python, JavaScript, ...)
- Bulleted lists
- CSV data
- Configuration files
- Any format where clean, copyable output matters

## Exercise

- Use message prefilling and stop sequences _only_ to get three different commands in a single response
- There shouldn't be any comments or explanations
- Hint: message prefilling isn't limited to just characters like ```

In [8]:
# code provided
messages = []

prompt = """
Generate three different sample AWS CLI commands. Each should be very short.
"""

add_user_message(messages, prompt)

text = chat(messages)
text.strip()

'Here are three different short AWS CLI commands:\n\n1. `aws s3 ls` - Lists all S3 buckets in your account\n\n2. `aws ec2 describe-instances` - Shows details of all EC2 instances\n\n3. `aws lambda list-functions` - Lists all Lambda functions in the current region'

In [9]:
from IPython.display import Markdown

Markdown(text)

Here are three different short AWS CLI commands:

1. `aws s3 ls` - Lists all S3 buckets in your account

2. `aws ec2 describe-instances` - Shows details of all EC2 instances

3. `aws lambda list-functions` - Lists all Lambda functions in the current region

In [11]:
# code provided
messages = []

prompt = """
Generate three different sample AWS CLI commands. Each should be very short.
"""

add_user_message(messages, prompt)
add_assistant_message(messages, "Here are the three commands in a single block without any comments: \n```bash")

text = chat(messages, stop_sequences=["```"])
text.strip()

'aws s3 ls\naws ec2 describe-instances\naws lambda list-functions'